### in this script, all kind of possible keys are investigated 

In [1]:
#### helper func to compare daq and phyiscs frames

def norm(x):
    return None if (isinstance(x, float) and math.isnan(x)) else x

In [2]:
from icecube import icetray, dataio, dataclasses, LeptonInjector
import math
from icecube.dataclasses import ModuleKey
from icecube.icetray import OMKey

In [3]:
DATA_PATH = '/project/def-nahee/kbas/trial_pom_response.i3.gz'
data_file = dataio.I3File(DATA_PATH)

In [4]:
data_file = dataio.I3File(DATA_PATH)
frame_daq = data_file.pop_frame()
frame_physics = data_file.pop_frame()

In [ ]:
Accepted_PulseMap_physics = frame_physics["Accepted_PulseMap"]
Accepted_PulseMap_daq = frame_daq["Accepted_PulseMap"]
####
PMT_Response_nonoise_physics = frame_physics["PMT_Response_nonoise"]
PMT_Response_nonoise_daq = frame_daq["PMT_Response_nonoise"]
# PMT seviyesinde ve sadece gercek pulse
####
PMT_Response_physics = frame_physics["PMT_Response"]
PMT_Response_daq = frame_daq["PMT_Response"]
# PMT seviyesinde ve noise + signal
####
EventPulseSeries_physics = frame_physics["EventPulseSeries"]
EventPulseSeries_daq = frame_daq["EventPulseSeries"]
####
EventPulseSeries_nonoise_physics = frame_physics["EventPulseSeries_nonoise"]
EventPulseSeries_nonoise_daq = frame_daq["EventPulseSeries_nonoise"]
####
Noise_K40_physics = frame_physics["Noise_K40"]
Noise_K40_daq = frame_daq["Noise_K40"]
####
Noise_Dark_physics = frame_physics["Noise_Dark"]
Noise_Dark_daq = frame_daq["Noise_Dark"]

In [6]:
n_daq = 0
n_physics = 0
n_other = {}

f = dataio.I3File(DATA_PATH)

while f.more():
    fr = f.pop_frame()
    stop = fr.Stop

    if stop == icetray.I3Frame.DAQ:
        n_daq += 1
    elif stop == icetray.I3Frame.Physics:
        n_physics += 1
    else:
        n_other[str(stop)] = n_other.get(str(stop), 0) + 1

f.close()

In [7]:
print("DAQ:", n_daq)
print("Physics:", n_physics)
print("Other:", n_other)

DAQ: 28
Physics: 28
Other: {}


In [8]:
print(frame_daq)

[ I3Frame  (DAQ):
  'Accepted_PulseMap' [DAQ] ==> I3Map<OMKey, vector<I3RecoPulse> > (2508)
  'DOMTrigger_HitTimes' [DAQ] ==> I3Map<OMKey, vector<double> > (942)
  'DOMTrigger_NCoin' [DAQ] ==> I3Map<OMKey, vector<int> > (203)
  'DOMTrigger_PMTs' [DAQ] ==> I3Map<OMKey, vector<int> > (527)
  'DOMTrigger_Time' [DAQ] ==> I3Map<OMKey, vector<double> > (294)
  'DetectorTriggers_3PMT_2DOM' [DAQ] ==> I3Vector<double> (38)
  'EventProperties' [DAQ] ==> LeptonInjector::BasicEventProperties (140)
  'EventPulseSeries' [DAQ] ==> I3Map<OMKey, vector<I3RecoPulse> > (145016)
  'EventPulseSeries_nonoise' [DAQ] ==> I3Map<OMKey, vector<I3RecoPulse> > (1462)
  'I3EventHeader' [DAQ] ==> I3EventHeader (99)
  'I3MCTree' [DAQ] ==> TreeBase::Tree<I3Particle, I3ParticleID, i3hash<I3ParticleID> > (422)
  'I3MCTree_RNGState' [DAQ] ==> I3GSLRandomServiceState (87)
  'I3MCTree_postprop' [DAQ] ==> TreeBase::Tree<I3Particle, I3ParticleID, i3hash<I3ParticleID> > (4026)
  'I3Photons' [DAQ] ==> I3Map<ModuleKey, I3Vector

In [9]:
print(frame_physics)

[ I3Frame  (Physics):
  'Accepted_PulseMap' [DAQ] ==> I3Map<OMKey, vector<I3RecoPulse> > (2508)
  'DOMTrigger_HitTimes' [DAQ] ==> I3Map<OMKey, vector<double> > (942)
  'DOMTrigger_NCoin' [DAQ] ==> I3Map<OMKey, vector<int> > (203)
  'DOMTrigger_PMTs' [DAQ] ==> I3Map<OMKey, vector<int> > (527)
  'DOMTrigger_Time' [DAQ] ==> I3Map<OMKey, vector<double> > (294)
  'DetectorTriggers_3PMT_2DOM' [DAQ] ==> I3Vector<double> (38)
  'EventProperties' [DAQ] ==> LeptonInjector::BasicEventProperties (140)
  'EventPulseSeries' [DAQ] ==> I3Map<OMKey, vector<I3RecoPulse> > (145016)
  'EventPulseSeries_nonoise' [DAQ] ==> I3Map<OMKey, vector<I3RecoPulse> > (1462)
  'I3EventHeader' [DAQ] ==> I3EventHeader (99)
  'I3MCTree' [DAQ] ==> TreeBase::Tree<I3Particle, I3ParticleID, i3hash<I3ParticleID> > (422)
  'I3MCTree_RNGState' [DAQ] ==> I3GSLRandomServiceState (87)
  'I3MCTree_postprop' [DAQ] ==> TreeBase::Tree<I3Particle, I3ParticleID, i3hash<I3ParticleID> > (4026)
  'I3Photons' [DAQ] ==> I3Map<ModuleKey, I3Ve

## 1- Accepted_PulseMap

In [10]:
same_nan_ok = all(
    [(p.time, p.charge, norm(p.width)) for p in Accepted_PulseMap_daq[k]] ==
    [(p.time, p.charge, norm(p.width)) for p in Accepted_PulseMap_physics[k]]
    for k in Accepted_PulseMap_physics.keys()
)


print("Accepted_PulseMap (DAQ) and Accepted_PulseMap (Physics) identical (if NaN treated equal):", same_nan_ok)


Accepted_PulseMap (DAQ) and Accepted_PulseMap (Physics) identical (if NaN treated equal): True


## 2- PMT_Response_nonoise

In [11]:
same_nan_ok = all(
    [(p.time, p.charge, p.width) for p in PMT_Response_nonoise_daq[k]] ==
    [(p.time, p.charge, p.width) for p in PMT_Response_nonoise_physics[k]]
    for k in PMT_Response_nonoise_physics.keys()
)


print("PMT_Response_nonoise (DAQ) and PMT_Response_nonoise (Physics) identical:", same_nan_ok)


PMT_Response_nonoise (DAQ) and PMT_Response_nonoise (Physics) identical: True


## 3- PMT_Response

In [12]:
same_nan_ok = all(
    [(p.time, p.charge, p.width) for p in PMT_Response_daq[k]] ==
    [(p.time, p.charge, p.width) for p in PMT_Response_physics[k]]
    for k in PMT_Response_physics.keys()
)


print("PMT_Response(DAQ) and PMT_Response (Physics) identical:", same_nan_ok)


PMT_Response(DAQ) and PMT_Response (Physics) identical: True


## 4- EventPulseSeries

In [13]:
same_nan_ok = all(
    [(p.time, p.charge, norm(p.width)) for p in EventPulseSeries_daq[k]] ==
    [(p.time, p.charge, norm(p.width)) for p in EventPulseSeries_physics[k]]
    for k in EventPulseSeries_physics.keys()
)


print("EventPulseSeries (DAQ) and EventPulseSeries (Physics) identical (if NaN treated equal):", same_nan_ok)

EventPulseSeries (DAQ) and EventPulseSeries (Physics) identical (if NaN treated equal): True


## 5- EventPulseSeries_nonoise 

In [14]:
same_nan_ok = all(
    [(p.time, p.charge, norm(p.width)) for p in EventPulseSeries_nonoise_daq[k]] ==
    [(p.time, p.charge, norm(p.width)) for p in EventPulseSeries_nonoise_physics[k]]
    for k in EventPulseSeries_nonoise_physics.keys()
)


print("EventPulseSeries_nonoise (DAQ) and EventPulseSeries_nonoise (Physics) identical (if NaN treated equal):", same_nan_ok)

EventPulseSeries_nonoise (DAQ) and EventPulseSeries_nonoise (Physics) identical (if NaN treated equal): True


## 6- Noise_K40

In [15]:
same_nan_ok = all(
    [(p.time, p.charge, p.width) for p in Noise_K40_daq[k]] ==
    [(p.time, p.charge, p.width) for p in Noise_K40_physics[k]]
    for k in Noise_K40_physics.keys()
)


print("Noise_K40 (DAQ) and Noise_K40 (Physics) identical:", same_nan_ok)


Noise_K40 (DAQ) and Noise_K40 (Physics) identical: True


## 7- Noise_Dark

In [16]:
same_nan_ok = all(
    [(p.time, p.charge, p.width) for p in Noise_Dark_daq[k]] ==
    [(p.time, p.charge, p.width) for p in Noise_Dark_physics[k]]
    for k in Noise_Dark_physics.keys()
)


print("Noise_Dark (DAQ) and Noise_Dark (Physics) identical:", same_nan_ok)


Noise_Dark (DAQ) and Noise_Dark (Physics) identical: True


In [17]:
# print("n keys:", len(Accepted_PulseMap_physics))     

In [18]:
#for k in Accepted_PulseMap_physics.keys():
#    print(k)

## 8- I3Photons

In [19]:
print(frame_daq['I3Photons'])

In [20]:
print(frame_physics['I3Photons'])

In [21]:
print(frame_daq.type_name("I3Photons"))
print(frame_physics.type_name("I3Photons"))


I3Map<ModuleKey, I3Vector<I3CompressedPhoton> >
I3Map<ModuleKey, I3Vector<I3CompressedPhoton> >


#### idk why, I3Photons do not work here. but it worked in clsim data. find out later

# Comparison Between EventPulseSeries and EventPulseSeries_nonoise

In [64]:
import bisect


def time_in_sorted_list(t, sorted_list, tol=1e-3):
    """t is in sorted_list within +/- tol? (sorted_list must be sorted)"""
    i = bisect.bisect_left(sorted_list, t)
    if i < len(sorted_list) and abs(sorted_list[i] - t) <= tol:
        return True
    if i > 0 and abs(sorted_list[i-1] - t) <= tol:
        return True
    return False

tol = 0  # ns (gerekirse 1e-2 yap)

missing = []  # list of (key, t) for nonoise times not found in noisy
stats = {
    "keys_in_nonoise": 0,
    "keys_missing_in_noisy": 0,
    "nonoise_pulses_total": 0,
    "matched_total": 0,
    "missing_total": 0,
}

for k in EventPulseSeries_nonoise_physics:
    stats["keys_in_nonoise"] += 1

    if k not in EventPulseSeries_physics:
        stats["keys_missing_in_noisy"] += 1
        for p in EventPulseSeries_nonoise_physics[k]:
            stats["nonoise_pulses_total"] += 1
            stats["missing_total"] += 1
            missing.append((k, p.time))
        continue

    noisy_times = sorted(p.time for p in EventPulseSeries_physics[k])

    for p in EventPulseSeries_nonoise_physics[k]:
        stats["nonoise_pulses_total"] += 1
        if time_in_sorted_list(p.time, noisy_times, tol=tol):
            stats["matched_total"] += 1
        else:
            stats["missing_total"] += 1
            missing.append((k, p.time))

print("=== Summary ===")
print("keys_in_nonoise:", stats["keys_in_nonoise"])
print("keys_missing_in_noisy:", stats["keys_missing_in_noisy"])
print("nonoise_pulses_total:", stats["nonoise_pulses_total"])
print("matched_total:", stats["matched_total"])
print("missing_total:", stats["missing_total"])
print("match_fraction:", stats["matched_total"] / max(1, stats["nonoise_pulses_total"]))




=== Summary ===
keys_in_nonoise: 26
keys_missing_in_noisy: 0
nonoise_pulses_total: 46
matched_total: 46
missing_total: 0
match_fraction: 1.0


In [62]:
missing

[]

In [63]:

n_keys_noisy   = len(EventPulseSeries_physics)
n_pulses_noisy = sum(len(EventPulseSeries_physics[k]) for k in EventPulseSeries_physics)

n_keys_nonoise   = len(EventPulseSeries_nonoise_physics)
n_pulses_nonoise = sum(len(EventPulseSeries_nonoise_physics[k]) for k in EventPulseSeries_nonoise_physics)

print("Noisy   : keys =", n_keys_noisy,   "| pulses =", n_pulses_noisy)
print("NoNoise : keys =", n_keys_nonoise, "| pulses =", n_pulses_nonoise)
print("Extra (Noisy - NoNoise): keys =", n_keys_noisy - n_keys_nonoise,
      "| pulses =", n_pulses_noisy - n_pulses_nonoise)


Noisy   : keys = 3733 | pulses = 3881
NoNoise : keys = 26 | pulses = 46
Extra (Noisy - NoNoise): keys = 3707 | pulses = 3835


In [22]:
### we have width in PMT_Response_nonoise but we dont have width in Accepted_PulseMap
### Accepted_PulseMap has charge 1 always

In [23]:
# print(frame['EventProperties'])

In [24]:
# print(frame['I3EventHeader'])

In [25]:
# print(frame['I3MCTree'])

In [26]:
# print(frame['I3MCTree_RNGState'])

In [27]:
# print(frame['I3MCTree_postprop'])

In [28]:
# print(frame['MMCTrackList'])

# Pulse Comparison

In [29]:
key1 = OMKey(282, 17, 10)  
key2 = OMKey(339, 20, 11)
key3 = OMKey(1  , 4,  10)

In [30]:
pulses_Accepted_PulseMap_physics = Accepted_PulseMap_physics[key1]
print(pulses_Accepted_PulseMap_physics)

[[I3RecoPulse:
             Time : 358.755
           Charge : 1
            Width : nan
            Flags : 
], [I3RecoPulse:
             Time : 356.031
           Charge : 1
            Width : nan
            Flags : 
]]


In [31]:
pulses_PMT_Response_nonoise_physics = PMT_Response_nonoise_physics[key1]
print(pulses_PMT_Response_nonoise_physics)

[[I3RecoPulse:
             Time : 353.276
           Charge : 0.770366
            Width : 10
            Flags : 
]]


In [32]:
pulses_PMT_Response_physics = PMT_Response_physics[key1]
print(pulses_PMT_Response_physics)

[[I3RecoPulse:
             Time : 353.276
           Charge : 1.35644
            Width : 10
            Flags : 
]]


###### neden Accepted_PulseMap'teki time, PMT_response'lardaki time'dan daha gec?

In [33]:
pulses_EventPulseSeries_physics = EventPulseSeries_physics[key1]
print(pulses_EventPulseSeries_physics)

[[I3RecoPulse:
             Time : 3750.28
           Charge : 1.35644
            Width : nan
            Flags : 
]]


In [34]:
pulses_EventPulseSeries_nonoise_physics = EventPulseSeries_nonoise_physics[key1]
print(pulses_EventPulseSeries_nonoise_physics)

[[I3RecoPulse:
             Time : 3750.28
           Charge : 0.770366
            Width : nan
            Flags : 
]]


In [35]:
pulses_Noise_K40_physics = Noise_K40_physics[key3]
print(pulses_Noise_K40_physics)

[[I3RecoPulse:
             Time : -990.217
           Charge : 1
            Width : 9
            Flags : 
]]


In [36]:
pulses_Noise_Dark_physics = Noise_Dark_physics[key2]
print(pulses_Noise_Dark_physics)

[[I3RecoPulse:
             Time : 5541.05
           Charge : 1
            Width : 10
            Flags : 
]]


In [37]:
## width bilgisi EventPulseSeries'a gecerken gidiyor

### Write down the meaning of these (they are same for DAQ and Physics):

In [ ]:
print(frame_daq['TriggerTime_3PMT_2DOM'])
print(frame_physics['TriggerTime_3PMT_2DOM'])
## bu DetectorTrigger ile uretilir.

# event trigger time, bu üç listedeki tüm zamanların en küçüğü (en erkeni):mintrigtime=min⁡(detectorTriggerTime∪stringTriggerTime∪singleOMTriggerTime)\text{mintrigtime} = \min(\text{detectorTriggerTime} \cup \text{stringTriggerTime} \cup \text{singleOMTriggerTime})mintrigtime=min(detectorTriggerTime∪stringTriggerTime∪singleOMTriggerTime)

I3Double(-1397)
I3Double(-1397)


In [ ]:
print(frame_daq['DOMTrigger_HitTimes'])
## DOM Trigger tarafinda (Detector trigger degil uretilir)

#  event trigger time (mintrigtime) PulseSeriesIn
# Trigger time’ı bulmak için kullandığı şeyler: `DOMTrigger_Time` + `DOMTrigger_NCoin` (ve geometri + pencere/eşik parametreleri). Bunlardan **detectorTriggerTime / stringTriggerTime / singleOMTriggerTime** listelerini üretiyor, sonra da bunların içinden **en erkenini** `mintrigtime` yapıyor.

# PulseSeriesIn ne için?: Trigger time bulundu **ktan sonra** sadece pulse’ları bir zaman penceresinde **kırpmak** ve zamanı `mintrigtime`’a göre **yeniden kaydırıp** `PulseSeriesOut` üretmek için kullanılıyor. Trigger time hesabına girmiyor.


[OMKey(264,17,0) => [484.234, 492.897, 490.501],
OMKey(265,16,0) => [117.084, 116.052, 117.269, 116.052, 117.269, 118.012, 117.084, 118.571, 119.463, 116.052, 117.269, 118.012, 117.084, 118.571, 119.463, 116.052, 117.269, 118.012, 121.025, 117.084, 118.571, 119.463, 118.012, 127.617, 121.025, 118.571, 127.617, 119.463, 128.694, 121.025, 119.463, 129.301, 127.617, 121.025, 128.694, 129.301, 137.501, 127.617, 128.694, 137.253, 137.501, 137.253, 129.301, 139.191, 137.501, 137.253, 140.215, 139.191, 137.501, 144.596, 140.215, 137.253, 139.191, 137.501, 144.596, 140.215, 145.116, 137.253, 139.191, 149.33, 144.596, 140.215, 145.116, 149.33, 144.596, 150.827, 145.116, 150.827, 149.33, 144.596, 152.015, 152.496, 145.116, 150.827, 149.33, 144.596, 153.458, 152.015, 152.496, 145.116],
OMKey(265,17,0) => [270.082, 270.955, 269.831, 270.346, 270.955, 270.082, 269.831, 271.297, 270.346, 271.544, 282.714, 287.267, 284.435, 282.714, 291.178, 287.267, 284.435],
OMKey(330,6,0) => [-1398.87, -1398.46, -

In [ ]:
print(frame_physics['DOMTrigger_HitTimes'])
## DOM Trigger tarafinda (Detector trigger degil uretilir)

[OMKey(264,17,0) => [484.234, 492.897, 490.501],
OMKey(265,16,0) => [117.084, 116.052, 117.269, 116.052, 117.269, 118.012, 117.084, 118.571, 119.463, 116.052, 117.269, 118.012, 117.084, 118.571, 119.463, 116.052, 117.269, 118.012, 121.025, 117.084, 118.571, 119.463, 118.012, 127.617, 121.025, 118.571, 127.617, 119.463, 128.694, 121.025, 119.463, 129.301, 127.617, 121.025, 128.694, 129.301, 137.501, 127.617, 128.694, 137.253, 137.501, 137.253, 129.301, 139.191, 137.501, 137.253, 140.215, 139.191, 137.501, 144.596, 140.215, 137.253, 139.191, 137.501, 144.596, 140.215, 145.116, 137.253, 139.191, 149.33, 144.596, 140.215, 145.116, 149.33, 144.596, 150.827, 145.116, 150.827, 149.33, 144.596, 152.015, 152.496, 145.116, 150.827, 149.33, 144.596, 153.458, 152.015, 152.496, 145.116],
OMKey(265,17,0) => [270.082, 270.955, 269.831, 270.346, 270.955, 270.082, 269.831, 271.297, 270.346, 271.544, 282.714, 287.267, 284.435, 282.714, 291.178, 287.267, 284.435],
OMKey(330,6,0) => [-1398.87, -1398.46, -

In [ ]:
print(frame_daq['DOMTrigger_NCoin'])
## DOM Trigger tarafinda (Detector trigger degil uretilir)

[OMKey(264,17,0) => [3],
OMKey(265,16,0) => [3, 5, 6, 7, 5, 4, 5, 5, 4, 4, 5, 6, 4, 4, 6, 7],
OMKey(265,17,0) => [4, 6, 3, 4],
OMKey(330,6,0) => [3]]


In [ ]:
print(frame_physics['DOMTrigger_NCoin'])
## DOM Trigger tarafinda (Detector trigger degil uretilir)

[OMKey(264,17,0) => [3],
OMKey(265,16,0) => [3, 5, 6, 7, 5, 4, 5, 5, 4, 4, 5, 6, 4, 4, 6, 7],
OMKey(265,17,0) => [4, 6, 3, 4],
OMKey(330,6,0) => [3]]


In [ ]:
print(frame_daq['DOMTrigger_Time'])
## DOM Trigger tarafinda (Detector trigger degil uretilir)

[OMKey(264,17,0) => [492],
OMKey(265,16,0) => [117, 118, 119, 121, 127, 128, 129, 137, 139, 140, 144, 145, 149, 150, 152, 153],
OMKey(265,17,0) => [270, 271, 287, 291],
OMKey(330,6,0) => [-1397]]


In [ ]:
print(frame_physics['DOMTrigger_Time'])
## DOM Trigger tarafinda (Detector trigger degil uretilir)

[OMKey(264,17,0) => [492],
OMKey(265,16,0) => [117, 118, 119, 121, 127, 128, 129, 137, 139, 140, 144, 145, 149, 150, 152, 153],
OMKey(265,17,0) => [270, 271, 287, 291],
OMKey(330,6,0) => [-1397]]


In [ ]:
print(frame_daq['DOMTrigger_PMTs'])
## DOM Trigger tarafinda (Detector trigger degil uretilir)

[OMKey(264,17,0) => [13, 12, 16],
OMKey(265,16,0) => [11, 7, 15, 7, 15, 2, 11, 16, 3, 7, 15, 2, 11, 16, 3, 7, 15, 2, 8, 11, 16, 3, 2, 11, 8, 16, 11, 3, 7, 8, 3, 16, 11, 8, 7, 16, 8, 11, 7, 15, 8, 15, 16, 7, 8, 15, 3, 7, 8, 4, 3, 15, 7, 8, 4, 3, 6, 15, 7, 15, 4, 3, 6, 15, 4, 16, 6, 16, 15, 4, 3, 11, 6, 16, 15, 4, 7, 3, 11, 6],
OMKey(265,17,0) => [9, 12, 11, 10, 12, 9, 11, 13, 10, 15, 11, 10, 3, 11, 12, 10, 3],
OMKey(330,6,0) => [14, 9, 12]]


In [ ]:
print(frame_physics['DOMTrigger_PMTs'])
## DOM Trigger tarafinda (Detector trigger degil uretilir)

[OMKey(264,17,0) => [13, 12, 16],
OMKey(265,16,0) => [11, 7, 15, 7, 15, 2, 11, 16, 3, 7, 15, 2, 11, 16, 3, 7, 15, 2, 8, 11, 16, 3, 2, 11, 8, 16, 11, 3, 7, 8, 3, 16, 11, 8, 7, 16, 8, 11, 7, 15, 8, 15, 16, 7, 8, 15, 3, 7, 8, 4, 3, 15, 7, 8, 4, 3, 6, 15, 7, 15, 4, 3, 6, 15, 4, 16, 6, 16, 15, 4, 3, 11, 6, 16, 15, 4, 7, 3, 11, 6],
OMKey(265,17,0) => [9, 12, 11, 10, 12, 9, 11, 13, 10, 15, 11, 10, 3, 11, 12, 10, 3],
OMKey(330,6,0) => [14, 9, 12]]


In [ ]:
print(frame_daq['StringTriggers_3PMT_2DOM'])
## bu DetectorTrigger ile uretilir.

# **String grubu**, geometriye göre oluşturulan bir *OM kümesi*: genelde **komşu string(ler)** üzerinde, belirli bir **OM katmanı aralığında (`StringNRows`)** yer alan OMLer bir grup yapılıyor ( `ForceAdjacency=True` ise sadece yeterince yakın stringler dahil ediliyor). String trigger, coincidence adaylarını bu grupların içinde topluca sayıyor.


# nasil uretilir:
# Önce coincidence ≥ OMPMTCoinc olan OM-zaman çiftleri aday yapılıyor; sonra aynı “string grubu” içinde, 
# StringCoincidenceWindow içinde farklı OM sayısı ≥ StringCoincidenceN olursa o grubun en geç zamanı stringTriggerTime’a yazılıyor.

[]


In [ ]:
print(frame_physics['StringTriggers_3PMT_2DOM'])
## bu DetectorTrigger ile uretilir.

# stringTrigger icinde sadece zaman var. format: [t1, t2, t3, ...]
# peki nasil uretiliyor (benim cumlelerimle): her bir OM icinde, >=2 PMT belli bir aralikta ise aday zamanlar cikariliyor. bu aday zamanlar, o OM'un ait oldugu gruplara dagitiliyor. boylece her grupta bissuru aday zamanlar oluyor.sonra, her grupta, belli bir zaman araliginda >2 tane OM'den gelen zaman varsa, bu gruptan trigger olur ve bu grupta temsilci bir zaman secilir. o zaman da stringTriggerTime listesine eklenir. cok yakin degerler varsa birlestirilir o degerler stringTriggerTime icinden.temsilci zaman da gruptaki max zaman olarak secilir.

[]


In [ ]:
print(frame_daq['DetectorTriggers_3PMT_2DOM'])
## bu DetectorTrigger ile uretilir.

## Yine coincidence ≥ OMPMTCoinc olan tüm OM-zaman adayları toplanıyor; FullDetectorCoincidenceWindow içinde 
# farklı OM sayısı ≥ FullDetectorCoincidenceN olursa en geç zaman detectorTriggerTime’a yazılıyor 
# (yakın trigger’lar EventLength içinde birleştiriliyor).

[]


In [ ]:
print(frame_physics['DetectorTriggers_3PMT_2DOM'])
## bu DetectorTrigger ile uretilir.

# nasil uretiliyor (benim cumlelerimle)
# detectorTriggerTime:durr. simdi, soyle oluyor surec. bos bir liste var. bu listede her OM'da >=2 pmt eger hit aldiysa, bunu o listeye eklyior. boylece o listede om id ve zaman var. sonra o zamani merkez alip belli bir zaman arligina bakiyor. o zaman araliginda (bir pencere) eger >=3 om hit aldiysa, penceredeki max zamani aliyor. bunu detectorTriggerTime'a ekliyor.

[]


In [ ]:
print(frame_daq['singleOMTrigger_3PMT_2DOM'])
## bu DetectorTrigger ile uretilir.

# nasil uretilir DOMTrigger_Time/NCoin içinden, tek bir OM içinde coincidence sayısı ≥ 3 olan zamanlar seçiliyor 
# ve bu zamanlar singleOMTrigger listesine ekleniyor.

[492, 117, 118, 119, 121, 127, 128, 129, 137, 139, 140, 144, 145, 149, 150, 152, 153, 270, 271, 287, 291, -1397]


In [ ]:
print(frame_physics['singleOMTrigger_3PMT_2DOM'])
## bu DetectorTrigger ile uretilir.

[492, 117, 118, 119, 121, 127, 128, 129, 137, 139, 140, 144, 145, 149, 150, 152, 153, 270, 271, 287, 291, -1397]


In [ ]:
print(frame_daq['triggerpulsemap'])
# OM seviyesinde response, ve signal + noise

[OMKey(1,4,0) => [[I3RecoPulse:
             Time : -984.085
           Charge : 1.14391
            Width : 10
            Flags : 
], [I3RecoPulse:
             Time : 5142.74
           Charge : 0.928871
            Width : 1
            Flags : 
]],
OMKey(1,6,0) => [[I3RecoPulse:
             Time : -1313.91
           Charge : 0.972108
            Width : 9
            Flags : 
], [I3RecoPulse:
             Time : 2260.14
           Charge : 1.05596
            Width : 4
            Flags : 
]],
OMKey(1,9,0) => [[I3RecoPulse:
             Time : 9404.01
           Charge : 0.662405
            Width : 5
            Flags : 
]],
OMKey(1,12,0) => [[I3RecoPulse:
             Time : 4315.59
           Charge : 1.40704
            Width : 15
            Flags : 
], [I3RecoPulse:
             Time : 7493.21
           Charge : 1.22439
            Width : 8
            Flags : 
]],
OMKey(1,13,0) => [[I3RecoPulse:
             Time : 3418.55
           Charge : 1.20982
            Width :

In [ ]:
print(frame_physics['triggerpulsemap'])
# OM seviyesinde response, ve signal + noise

[OMKey(1,4,0) => [[I3RecoPulse:
             Time : -984.085
           Charge : 1.14391
            Width : 10
            Flags : 
], [I3RecoPulse:
             Time : 5142.74
           Charge : 0.928871
            Width : 1
            Flags : 
]],
OMKey(1,6,0) => [[I3RecoPulse:
             Time : -1313.91
           Charge : 0.972108
            Width : 9
            Flags : 
], [I3RecoPulse:
             Time : 2260.14
           Charge : 1.05596
            Width : 4
            Flags : 
]],
OMKey(1,9,0) => [[I3RecoPulse:
             Time : 9404.01
           Charge : 0.662405
            Width : 5
            Flags : 
]],
OMKey(1,12,0) => [[I3RecoPulse:
             Time : 4315.59
           Charge : 1.40704
            Width : 15
            Flags : 
], [I3RecoPulse:
             Time : 7493.21
           Charge : 1.22439
            Width : 8
            Flags : 
]],
OMKey(1,13,0) => [[I3RecoPulse:
             Time : 3418.55
           Charge : 1.20982
            Width :